In [2]:
import pandas as pd
import random
import string








In [9]:
import pandas as pd
import random
import string
import sqlite3
from sqlite3 import Error
from faker import Faker

def create_connection(db_file):
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(f'successful connection with sqlite version {sqlite3.version}')
    except Error as e:
        print(e)
    return conn

def create_table(conn, table_name, columns):
    try:
        cur = conn.cursor()
        column_str = ', '.join([f'{col} {ctype}' for col, ctype in columns.items()])
        cur.execute(f'CREATE TABLE IF NOT EXISTS {table_name} ({column_str});')
        print(f'Table {table_name} created.')
    except Error as e:
        print(e)

def insert_data(conn, table_name, data):
    try:
        cur = conn.cursor()
        column_names = ', '.join(data.columns)
        placeholders = ', '.join(['?' for _ in data.columns])
        cur.executemany(f'INSERT INTO {table_name} ({column_names}) VALUES ({placeholders})', data.values.tolist())
        conn.commit()
        print(f'{cur.rowcount} rows inserted into {table_name}.')
    except Error as e:
        print(e)

def main():
    # Create a connection to a SQLite database
    conn = create_connection('bikerental.db')

    # Create tables
    table_columns = {
        'tblusers': {
            'id': 'INTEGER PRIMARY KEY',
            'FullName': 'TEXT NOT NULL',
            'EmailId': 'TEXT UNIQUE NOT NULL'
        },
        'tblvehicles': {
            'id': 'INTEGER PRIMARY KEY',
            'VehiclesTitle': 'TEXT NOT NULL',
            'PricePerDay': 'INTEGER NOT NULL'
        },
        'tblbooking': {
            'id': 'INTEGER PRIMARY KEY',
            'userEmail': 'TEXT NOT NULL',
            'VehicleId': 'INTEGER NOT NULL',
            'FromDate': 'TEXT NOT NULL',
            'ToDate': 'TEXT NOT NULL'
        }
    }


In [10]:
def main():
    # Create a connection to a SQLite database
    conn = create_connection('bikerental.db')

    # Create tables
    table_columns = {
        # ... (The table_columns dictionary remains the same)
    }

    for table_name, columns in table_columns.items():
        create_table(conn, table_name, columns)

    # Generate synthetic data
    fake = Faker()
    users = pd.DataFrame({
        'id': range(1, 1001),
        'FullName': [fake.name() for _ in range(1000)],
        'EmailId': [fake.email() for _ in range(1000)]
    })

    bikes = pd.DataFrame({
        'id': range(1, 101),
        'VehiclesTitle': [f'Bike_{i}' for i in range(1, 101)],
        'PricePerDay': random.sample(range(50, 200), 100)
    })

    bookings = pd.DataFrame({
        'id': range(1, 101),
        'userEmail': [random.choice(users['EmailId']) for _ in range(100)],
        'VehicleId': [random.choice(bikes['id']) for _ in range(100)],
        'FromDate': pd.date_range(start='2021-01-01', periods=100, freq='D').strftime('%Y-%m-%d').tolist(),
        'ToDate': pd.date_range(start='2021-01-02', periods=100, freq='D').strftime('%Y-%m-%d').tolist()
    })

    # Insert data into the tables
    insert_data(conn, 'tblusers', users)
    insert_data(conn, 'tblvehicles', bikes)
    insert_data(conn, 'tblbooking', bookings)

    # Close the connection
    conn.close()

if __name__ == '__main__':
    main()

successful connection with sqlite version 2.6.0
no such table: tblusers
no such table: tblvehicles
no such table: tblbooking


In [11]:
import sqlite3
import pandas as pd
from faker import Faker
import random

def create_connection(db_file):
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(f"Successfully Connected to SQLite")
    except Error as e:
        print(e)
    return conn

def create_table(conn, create_table_sql):
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)

def insert_data(conn, table_name, df):
    try:
        df.to_sql(table_name, conn, if_exists='append', index=False)
        print(f"Successfully Inserted Data into {table_name}")
    except Error as e:
        print(e)

# Create a connection to a SQLite database
conn = create_connection('bikerental.db')

# Create tables
table_columns = {
    'tblusers': ['id INTEGER PRIMARY KEY', 'FullName TEXT', 'EmailId TEXT'],
    'tblvehicles': ['id INTEGER PRIMARY KEY', 'VehiclesTitle TEXT', 'PricePerDay INTEGER'],
    'tblbooking': ['id INTEGER PRIMARY KEY', 'userEmail TEXT', 'VehicleId INTEGER', 'FromDate DATE', 'ToDate DATE']
}

for table_name, columns in table_columns.items():
    create_table_sql = f"CREATE TABLE IF NOT EXISTS {table_name} ({', '.join(columns)});"
    create_table(conn, create_table_sql)

# Generate synthetic data
fake = Faker()
users = pd.DataFrame({
    'id': range(1, 1001),
    'FullName': [fake.name() for _ in range(1000)],
    'EmailId': [fake.email() for _ in range(1000)]
})

bikes = pd.DataFrame({
    'id': range(1, 101),
    'VehiclesTitle': [f'Bike_{i}' for i in range(1, 101)],
    'PricePerDay': random.sample(range(50, 200), 100)
})

bookings = pd.DataFrame({
    'id': range(1, 101),
    'userEmail': [random.choice(users['EmailId']) for _ in range(100)],
    'VehicleId': [random.choice(bikes['id']) for _ in range(100)],
    'FromDate': pd.date_range(start='2021-01-01', periods=100, freq='D').strftime('%Y-%m-%d').tolist(),
    'ToDate': pd.date_range(start='2021-01-02', periods=100, freq='D').strftime('%Y-%m-%d').tolist()
})

# Insert data into the tables
insert_data(conn, 'tblusers', users)
insert_data(conn, 'tblvehicles', bikes)
insert_data(conn, 'tblbooking', bookings)

# Close the connection
conn.close()

Successfully Connected to SQLite
Successfully Inserted Data into tblusers
Successfully Inserted Data into tblvehicles
Successfully Inserted Data into tblbooking


In [12]:
import sqlite3
import pandas as pd

# Create a connection to the SQLite database
conn = sqlite3.connect('bikerental.db')

# Function to save a table as a CSV file
def save_table_as_csv(table_name, csv_file):
    df = pd.read_sql_query(f"SELECT * FROM {table_name}", conn)
    df.to_csv(csv_file, index=False)
    print(f"Successfully Saved {table_name} as {csv_file}")

# Save the tables as CSV files
save_table_as_csv('tblusers', 'tblusers.csv')
save_table_as_csv('tblvehicles', 'tblvehicles.csv')
save_table_as_csv('tblbooking', 'tblbooking.csv')

# Close the connection
conn.close()

Successfully Saved tblusers as tblusers.csv
Successfully Saved tblvehicles as tblvehicles.csv
Successfully Saved tblbooking as tblbooking.csv
